In [2]:
import os


In [3]:
%pwd

'/Users/vivek/work/GIT/end-to-end-mlflow/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/vivek/work/GIT/end-to-end-mlflow'

### defining the entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

### adding the configuration manager


In [7]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_training_config(self)-> TrainingConfig:
        training = self.config.training
        prepare_base_modle = self.config.prepare_base_model
        params = self.params
        trainng_data = os.path.join(self.config.data_ingestion.unzip_dir, "chest-ct-scan-data")
        create_directories(
            Path(training.root_dir)
        )

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_modle.updated_base_model_path),
            trainng_data=Path(trainng_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config
    

In [11]:
import os 
import urllib.request as req
from zipapp import zipfile
import tensorflow as tf
import time


In [16]:
class Training:
    def __init__(self, config = TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generate(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **dataflow_kwargs
            )

        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.step_per_epochs = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            step_per_epochs = self.step_per_epochs,
            validation_step = self.validation_steps,
            validation_data = self.valid_generator
        )

        self.save_model(
            path = self.config.trained_model_path,
            model=self.model
        )

### creating the pipeline

In [17]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generate()
    training.train()
except Exception as e:
    raise e

[2024-09-23 10:22:38,551 : INFO : common : yaml file: config/config.yaml loaded successfully]
[2024-09-23 10:22:38,554 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-09-23 10:22:38,555 : INFO : common : created directory at: artifacts]


EnsureError: Argument path_to_directories of type <class 'pathlib.PosixPath'> to <function create_directories at 0x1067f7520> does not match annotation type <class 'list'>